In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [2]:
SEED = 1

# CUDA?
# cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("CUDA Available?", device)

# For reproducibility
torch.manual_seed(SEED)

if device:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=64, num_workers=10, pin_memory=True) if device else dict(shuffle=True, batch_size=4)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset,** dataloader_args)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, **dataloader_args)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

CUDA Available? cuda:0
Files already downloaded and verified
Files already downloaded and verified


In [3]:
class ResNet18(torch.nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()

        self.input_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1), # input: 3x3 output: 32 Rec.field: 1 JumpOut: 1
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

        self.x1 =nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1), # input: 3x3 output: 32 Rec.field: 1 JumpOut: 1
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

        self.R1 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1), # input: 3x3 output: 16 Rec.field: 1 JumpOut: 1
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1), # input: 3x3 output: 16 Rec.field: 1 JumpOut: 1
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

#         self.layer1=nn.Sequential(
#             nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1), # input: 3x3 output: 16 Rec.field: 1 JumpOut: 1
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
#         )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1),# input: 3x3 output: 16 Rec.field: 1 JumpOut: 1
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )

        self.x2 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1), # input: 3x3 output: 8 Rec.field: 1 JumpOut: 1
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )

        self.R2 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1), # input: 3x3 output: 4 Rec.field: 1 JumpOut: 1
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1), # input: 3x3 output: 4 Rec.field: 1 JumpOut: 1
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=10, kernel_size=3, stride=1, padding=1),# input: 3x3 output: 4 Rec.field: 1 JumpOut: 1
            nn.BatchNorm2d(10),
            nn.ReLU(),
        )

        self.pool1 = nn.MaxPool2d(4, 4)
        self.linear = nn.Linear(1 , 10)

    def forward(self, x):
        out= self.input_layer(x);
        X = self.x1(out)
        R = self.R1(X)
        out = self.x1(out)+self.R1(X)
        out = self.layer2(out)
        X = self.x2(out)
        R = self.R2(X)
        out = self.x2(out) + self.R2(X)
        out = self.pool1(out) 
        out = self.linear(out)
#         return out
        # out = out.view(-1, 10)
        return F.log_softmax(out, dim=-1)


In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
net = ResNet18().to(device)
summary(net, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,928
         MaxPool2d-5           [-1, 64, 16, 16]               0
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,928
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]          36,928
      BatchNorm2d-12           [-1, 64, 16, 16]             128
             ReLU-13           [-1, 64, 16, 16]               0
           Conv2d-14           [-1

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(net, device, optimizer,criterion, epoch,trainloader):
  #trainloader = dataloader_cifar10(root='./data',, split='train', batch_size=args.batch_size)
  net.train()
  pbar = tqdm(trainloader)
  correct = 0
  processed = 0
  for batch_idx, (input, labels) in enumerate(pbar):
    # get samples
    input, labels = input.to(device), labels.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    outputs = net(input)

    # Calculate loss
    loss = F.nll_loss(outputs, labels)
#     loss = criterion(outputs, labels)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    _, pred = torch.max(outputs, 1)  # get the index of the max log-probability
    correct += pred.eq(labels.view_as(pred)).sum().item()
    processed += len(input)

    pbar.set_description(desc= f'Epoch={epoch} Loss={loss.item()} Batch_id={batch_idx} Training Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)
    

def test(net, device,criterion, testloader):
    #testloader = dataloader_cifar10(root='./data',, split='test', batch_size=dataloader_args.batch_size)
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    accuracy = 0
    classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
#             loss = criterion(outputs, targets)
            loss = F.nll_loss(outputs, target)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            #     % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        test_loss /= len(testloader.dataset)
                            
        if accuracy >=90:
            class_correct = list(0. for i in range(10))
            class_total = list(0. for i in range(10))
            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs, 1)
                    c = (predicted == labels).squeeze()
                    for i in range(4):
                        label = labels[i]
                        class_correct[label] += c[i].item()
                        class_total[label] += 1


            for i in range(10):
                print('Accuracy of %5s : %2d %%' % (
                    classes[i], 100 * class_correct[i] / class_total[i]))
                
        accuracy=100.00 * correct / len(testloader.dataset)
        
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                test_loss, correct, len(testloader.dataset),
                accuracy))

In [9]:
net =  ResNet18().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 2
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(net, device, optimizer,criterion, epoch,trainloader)
    test(net, device,criterion, testloader)

EPOCH: 0



  0%|                                                                                          | 0/782 [00:00<?, ?it/s]

RuntimeError: 1only batches of spatial targets supported (non-empty 3D tensors) but got targets of size: : [64]